In [ ]:
import sys
import os
from pathlib import Path
from glob import glob
import pandas as pd
import json
from ast import literal_eval
import numpy as np
from classes.literature import literature
lit = literature()

## GET SELF-RESISTANCE FULLTEXT SECTIONS

In [ ]:
srdf = pd.read_csv('data/abstracts_self_resistance_and_gene.csv')
srdf['pmid'] = srdf['pmid'].apply(lambda x: str(x)) # need to convert to string for europepmc query
srpmids = list(srdf['pmid'])

#### GET PMCIDS FOR PMIDS

In [ ]:
lit.xrf_pmc('self_resistance','data',srpmids)

#### GET PMC XMLS FROM EUROPEPMC API

In [ ]:
lit.get_ft_xml('data/pmc_xml','data/self_resistance_pmcids.json')

#### PARSE PMC XML AND EXTRACT SECIONS TEXTS

In [ ]:
lit.get_ft_text('data/pmc_xml',['data/pmc_txt','data/pmc_json'],'data/self_resistance_pmcids.json')